In [239]:
!pip install --upgrade --no-cache-dir scikit-learn==1.7.1

In [240]:
import sklearn
!python --version
print(f"\nSklearn {sklearn.__version__}")

Python 3.12.7

Sklearn 1.7.1


In [241]:
# Importing important libraries
import pandas as pd
import pymysql
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import pickle
import warnings
warnings.filterwarnings("ignore")

In [242]:
# 1.1. Creating connection to MySQL Database
connection = pymysql.connect(
    host = 'localhost', 
    user = 'root',
    password = 'Gautham@123', # Use your own password here
    database = 'BigMart'
)

In [243]:
cursor = connection.cursor()

In [244]:
# -----------------------------
# 1.2. Create Database
# -----------------------------
cursor.execute("DROP DATABASE IF EXISTS loan_db")
cursor.execute("CREATE DATABASE loan_db")
cursor.execute("USE loan_db")

0

In [245]:
# -----------------------------
# 1.3. Create Tables
# -----------------------------
cursor.execute("""
CREATE TABLE IF NOT EXISTS applicant_info (
    Loan_ID VARCHAR(20) PRIMARY KEY,
    Gender VARCHAR(10),
    Married VARCHAR(10),
    Dependents VARCHAR(5),
    Education VARCHAR(50),
    Self_Employed VARCHAR(10)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS financial_info (
    Loan_ID VARCHAR(20) PRIMARY KEY,
    ApplicantIncome INT,
    CoapplicantIncome FLOAT,
    LoanAmount FLOAT,
    Loan_Amount_Term FLOAT,
    Credit_History FLOAT,
    FOREIGN KEY (Loan_ID) REFERENCES applicant_info(Loan_ID)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS loan_info (
    Loan_ID VARCHAR(20) PRIMARY KEY,
    Property_Area VARCHAR(20),
    Loan_Status VARCHAR(5),
    FOREIGN KEY (Loan_ID) REFERENCES applicant_info(Loan_ID)
)
""")

0

In [246]:
# -----------------------------
# 1.4. Load JSON Files
# -----------------------------
def load_json_to_table(filename, table, fields):
    with open(filename, "r") as f:
        for line in f:
            record = json.loads(line)
            values = [record.get(field) for field in fields]
            placeholders = ", ".join(["%s"] * len(values))
            sql = f"REPLACE INTO {table} ({', '.join(fields)}) VALUES ({placeholders})"
            cursor.execute(sql, values)

In [247]:
# Insert applicant_info
load_json_to_table(
    "applicant_info.json",
    "applicant_info",
    ["Loan_ID", "Gender", "Married", "Dependents", "Education", "Self_Employed"]
)

In [248]:
# Creating the data for applicant
df_applicant = pd.read_sql("SELECT * FROM applicant_info",connection)
df_applicant.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed
0,LP001002,Male,No,0,Graduate,No
1,LP001003,Male,Yes,1,Graduate,No
2,LP001005,Male,Yes,0,Graduate,Yes
3,LP001006,Male,Yes,0,Not Graduate,No
4,LP001008,Male,No,0,Graduate,No


In [249]:
# Insert financial_info
load_json_to_table(
    "financial_info.json",
    "financial_info",
    ["Loan_ID", "ApplicantIncome", "CoapplicantIncome", "LoanAmount", "Loan_Amount_Term", "Credit_History"]
)

In [250]:
# Creating the data for financial
df_financial = pd.read_sql("SELECT * FROM financial_info",connection)
df_financial.head()

,Loan_ID,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
0,LP001002,5849,0.0,NaN,360.0,1.0
1,LP001003,4583,1508.0,128.0,360.0,1.0
2,LP001005,3000,0.0,66.0,360.0,1.0
3,LP001006,2583,2358.0,120.0,360.0,1.0
4,LP001008,6000,0.0,141.0,360.0,1.0


In [251]:
# Insert loan_info
load_json_to_table(
    "loan_info.json",
    "loan_info",
    ["Loan_ID", "Property_Area", "Loan_Status"]
)

In [252]:
# Creating the data for loan
df_financial = pd.read_sql("SELECT * FROM loan_info",connection)
df_financial.head()

,Loan_ID,Property_Area,Loan_Status
0,LP001002,Urban,Y
1,LP001003,Rural,N
2,LP001005,Urban,Y
3,LP001006,Urban,Y
4,LP001008,Urban,Y


In [253]:
# -----------------------------
# 5. Commit
# -----------------------------
connection.commit()

In [254]:
'''
2. Data Retrieval & Preprocessing
Objective: Prepare data for training.
Tasks:
● Retrieve data from MySQL into Pandas DataFrames.
● Merge datasets into a single table using a common key (e.g., Applicant_ID).
● Handle missing values.
● Encode categorical features (Gender, Education, Marital Status, etc.).
● Scale numerical features if necessary.
● Split the dataset into train/test sets.
'''

'\n2. Data Retrieval & Preprocessing\nObjective: Prepare data for training.\nTasks:\n● Retrieve data from MySQL into Pandas DataFrames.\n● Merge datasets into a single table using a common key (e.g., Applicant_ID).\n● Handle missing values.\n● Encode categorical features (Gender, Education, Marital Status, etc.).\n● Scale numerical features if necessary.\n● Split the dataset into train/test sets.\n'

In [255]:
# 2.1. Retrieve data from MySQL into Pandas DataFrames.
query = """
SELECT 
    a.Loan_ID, a.Gender, a.Married, a.Dependents, a.Education, a.Self_Employed,
    f.ApplicantIncome, f.CoapplicantIncome, f.LoanAmount, f.Loan_Amount_Term, f.Credit_History,
    l.Property_Area, l.Loan_Status
FROM applicant_info a
JOIN financial_info f ON a.Loan_ID = f.Loan_ID
JOIN loan_info l ON a.Loan_ID = l.Loan_ID
"""
df = pd.read_sql(query, connection)
connection.close()
cursor.close()
print("✅ Data Retrieved from MySQL")
print(df.head())

✅ Data Retrieved from MySQL
    Loan_ID Gender Married Dependents     Education Self_Employed  \
0  LP001002   Male      No          0      Graduate            No   
1  LP001003   Male     Yes          1      Graduate            No   
2  LP001005   Male     Yes          0      Graduate           Yes   
3  LP001006   Male     Yes          0  Not Graduate            No   
4  LP001008   Male      No          0      Graduate            No   

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5849                0.0         NaN             360.0   
1             4583             1508.0       128.0             360.0   
2             3000                0.0        66.0             360.0   
3             2583             2358.0       120.0             360.0   
4             6000                0.0       141.0             360.0   

   Credit_History Property_Area Loan_Status  
0             1.0         Urban           Y  
1             1.0         Rural       

In [256]:
# Check NULL counts per column
print(df.isnull().sum())

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64


In [257]:
# Getting the idea about the shape of the data
print("Shape :", df.shape)
print("Number of rows :", df.shape[0])
print("Number of columns :", df.shape[1])

Shape : (614, 13)
Number of rows : 614
Number of columns : 13


In [258]:
# -----------------------------
# 2.2. Handle Missing Values
# -----------------------------
# (a) CoapplicantIncome → if Married == "No", set missing to 0
df.loc[(df["Married"] == "No") & (df["CoapplicantIncome"].isna()), "CoapplicantIncome"] = 0

# (b) LoanAmount → fill ONLY nulls with median grouped by (Education, Self_Employed, Property_Area)
df.loc[df["LoanAmount"].isna(), "LoanAmount"] = df.groupby(
    ["Education", "Self_Employed", "Property_Area"]
)["LoanAmount"].transform(lambda x: x.fillna(x.median()))

#  Global median fallback (for groups where all values were NaN)
df["LoanAmount"].fillna(df["LoanAmount"].median(), inplace=True)

# (c) Loan_Amount_Term → most common (mode, usually 360 months)
df["Loan_Amount_Term"].fillna(df["Loan_Amount_Term"].mode()[0], inplace=True)

# (d) Credit_History → introduce new category = 2 for Unknown
df["Credit_History"].fillna(2, inplace=True)

# (e) Handle Married column
# If Dependents > 0 or CoapplicantIncome > 0 → Married = "Yes"
df.loc[df["Married"].isna() & (df["Dependents"].astype(str) != "0"), "Married"] = "Yes"
df.loc[df["Married"].isna() & (df["CoapplicantIncome"] > 0), "Married"] = "Yes"

# If Dependents == 0 and CoapplicantIncome == 0 → Married = "No"
df.loc[df["Married"].isna() & (df["Dependents"].astype(str) == "0") & (df["CoapplicantIncome"] == 0), "Married"] = "No"

# Fallback → fill remaining nulls with mode
mode_married = df["Married"].mode()[0]
df["Married"].fillna(mode_married, inplace=True)

# (f) Dependents → if Married == "No", set missing to 0; else fill with mode
df.loc[(df["Married"] == "No") & (df["Dependents"].isna()), "Dependents"] = "0"
df["Dependents"].fillna(df["Dependents"].mode()[0], inplace=True)

# (g) Self_Employed → fill with mode
df["Self_Employed"].fillna(df["Self_Employed"].mode()[0], inplace=True)

# (h) Gender → fill with mode
df["Gender"].fillna(df["Gender"].mode()[0], inplace=True)

# (i) Property_Area → fill with mode
df["Property_Area"].fillna(df["Property_Area"].mode()[0], inplace=True)

# (j) Loan_Status (Target) → drop rows if missing (since we can’t train without labels)
df = df.dropna(subset=["Loan_Status"])

print("✅ Business Rules Applied for Missing Values")
print(df.isnull().sum())

✅ Business Rules Applied for Missing Values
Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64


In [259]:
# -----------------------------
# 2.3. Encode Categorical Features
# -----------------------------
# Binary categorical → LabelEncoder
binary_cols = ["Gender", "Married", "Education", "Self_Employed", "Loan_Status"]
label_encoders = {}
for col in binary_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Multi-category categorical → OneHotEncoder
multi_cat_cols = ["Dependents", "Property_Area"]
df = pd.get_dummies(df, columns=multi_cat_cols, drop_first=True)

print("✅ Encoding Done (LabelEncoder + OneHotEncoder)")

✅ Encoding Done (LabelEncoder + OneHotEncoder)


In [260]:
# -----------------------------
# 2.4. Scale Numerical Features
# -----------------------------
num_cols = ["ApplicantIncome", "CoapplicantIncome", "LoanAmount", "Loan_Amount_Term", "Credit_History"]

scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])
print("✅ Numerical Features Scaled")

✅ Numerical Features Scaled


In [261]:
# -----------------------------
# 5. Train/Test Split
# -----------------------------
X = df.drop(columns=["Loan_ID", "Loan_Status"])
y = df["Loan_Status"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("✅ Data Split into Train/Test Sets")
print("Train shape:", X_train.shape, "Test shape:", X_test.shape)

✅ Data Split into Train/Test Sets
Train shape: (491, 14) Test shape: (123, 14)


In [262]:
'''
3. Model Training
Objective: Build a classification model for loan approval prediction.
Tasks:
● Train a RandomForestClassifier (or other suitable ML algorithm).
● Evaluate using accuracy, precision, recall, and F1-score.
● Save trained model as model.pkl using pickle.
'''

'\n3. Model Training\nObjective: Build a classification model for loan approval prediction.\nTasks:\n● Train a RandomForestClassifier (or other suitable ML algorithm).\n● Evaluate using accuracy, precision, recall, and F1-score.\n● Save trained model as model.pkl using pickle.\n'

In [263]:
# -----------------------------
# 3.1. Train Model
# -----------------------------
rf_clf = RandomForestClassifier(
    n_estimators=200,       # number of trees
    max_depth=None,        # let trees grow fully
    random_state=42,
    class_weight="balanced"  # handle class imbalance
)

rf_clf.fit(X_train, y_train)
print("✅ RandomForest Model Trained")

✅ RandomForest Model Trained


In [264]:
# -----------------------------
# 2. Evaluate Model
# -----------------------------
y_pred = rf_clf.predict(X_test)

accuracy  = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall    = recall_score(y_test, y_pred)
f1        = f1_score(y_test, y_pred)

print("\n=== Model Evaluation Metrics ===")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1-score : {f1:.4f}")
print("\nDetailed Report:")
print(classification_report(y_test, y_pred))


=== Model Evaluation Metrics ===
Accuracy : 0.8374
Precision: 0.8421
Recall   : 0.9412
F1-score : 0.8889

Detailed Report:
              precision    recall  f1-score   support

           0       0.82      0.61      0.70        38
           1       0.84      0.94      0.89        85

    accuracy                           0.84       123
   macro avg       0.83      0.77      0.79       123
weighted avg       0.84      0.84      0.83       123



In [265]:
# -----------------------------
# 3. Save Model
# -----------------------------
with open("model.pkl", "wb") as f:
    pickle.dump(rf_clf, f)

print("✅ Trained model saved as model.pkl")

✅ Trained model saved as model.pkl
